In [5]:
# Load .npy files
import numpy as np

# Load features and labels
train_manual    = np.load("featuresets/train_manual_features.npy")
test_manual     = np.load("featuresets/test_manual_features.npy")
train_mobilenet = np.load("featuresets/train_mobilenet_features.npy")
test_mobilenet  = np.load("featuresets/test_mobilenet_features.npy")
train_densenet  = np.load("featuresets/train_densenet_features.npy")
test_densenet   = np.load("featuresets/test_densenet_features.npy")
train_labels    = np.load("featuresets/train_manual_labels.npy")
test_labels     = np.load("featuresets/test_manual_labels.npy")


In [7]:
# ✅ Combine features
X_train = np.hstack((train_manual, train_mobilenet, train_densenet))
X_test  = np.hstack((test_manual, test_mobilenet, test_densenet))

print("Train shape:", X_train.shape)
print("Test shape:", X_test.shape)
print("Labels:", train_labels.shape)

# 🔍 Remove Class 6
class_to_remove = 6
train_mask = train_labels != class_to_remove
test_mask  = test_labels != class_to_remove

X_train_filtered = X_train[train_mask]
X_test_filtered  = X_test[test_mask]
y_train_filtered = train_labels[train_mask]
y_test_filtered  = test_labels[test_mask]

print("Filtered train shape:", X_train_filtered.shape)
print("Filtered test shape:", X_test_filtered.shape)

# 🤖 Import models
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, accuracy_score

# 🧠 3-Model Ensemble (SVM + XGB + RF)
ensemble = VotingClassifier(
    estimators=[
        ('svm', SVC(probability=True, C=10, gamma='scale', random_state=42)),
        ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)),
        ('rf', RandomForestClassifier(n_estimators=150, random_state=42))
    ],
    voting='soft',
    n_jobs=-1
)

# 🏋️ Train and Predict
ensemble.fit(X_train_filtered, y_train_filtered)
y_pred = ensemble.predict(X_test_filtered)

# 📊 Evaluate
print("\n🤝 Ensemble Voting Results (Top 3 Models)")
print(classification_report(y_test_filtered, y_pred))
print("Voting Ensemble Accuracy:", accuracy_score(y_test_filtered, y_pred))

Train shape: (8400, 2318)
Test shape: (1400, 2318)
Labels: (8400,)
Filtered train shape: (6900, 2318)
Filtered test shape: (1200, 2318)

🤝 Ensemble Voting Results (Top 3 Models)
              precision    recall  f1-score   support

           0       0.78      0.76      0.77       200
           1       0.88      0.93      0.90       200
           2       0.78      0.80      0.79       200
           3       0.86      0.82      0.84       200
           4       0.90      0.91      0.90       200
           5       0.81      0.81      0.81       200

    accuracy                           0.84      1200
   macro avg       0.84      0.84      0.84      1200
weighted avg       0.84      0.84      0.84      1200

Voting Ensemble Accuracy: 0.8366666666666667


In [8]:
import joblib

# Save the ensemble model
joblib.dump(ensemble, "ensemble_model.joblib")
print("✅ Ensemble model saved successfully!")

✅ Ensemble model saved successfully!
